In [2]:
import importlib
import sqlite3
import pandas as pd
from lib import kpi

In [3]:
importlib.reload(kpi)

<module 'lib.kpi' from 'c:\\workspace\\GitHub\\arcsaef\\lib\\kpi.py'>

In [4]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, data[2], rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

In [197]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say' or ppl_saef[prsn]['Gender'] == 'Non-binary':
        ppl_saef[prsn]['Gender'] = 'Unspecified diversity'
# 3. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 4. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

In [198]:
active_ppl_saef = {}
for p in ppl_saef:
    if ppl_saef[p]['State'] == 'Active' and ppl_saef[p]['Gender'] != '':
        if ppl_saef[p]['Position'] in [ 'Research Professional', 'PhD Student', 'Chief Investigator', 'Partner Investigator', 'Post Doc']:
            position = 'Researcher'
        else: 
            position = ppl_saef[p]['Position']

        active_ppl_saef[p] = {
            'FirstName':    ppl_saef[p]['FirstName'] , 
            'LastName':     ppl_saef[p]['LastName'],
            'Gender':       ppl_saef[p]['Gender'],
            'Position':     position}       


In [199]:
positions = []
for p in active_ppl_saef:
    if active_ppl_saef[p]['Position'] in ['Program Staff', 'Researcher']:
        row = [active_ppl_saef[p]['FirstName'], active_ppl_saef[p]['LastName'], active_ppl_saef[p]['Gender'], active_ppl_saef[p]['Position'] ]
        positions.append(row)

In [200]:
for p in active_ppl_saef:
    for s in responses_json['People_Detail']['data']:
        if s['fieldData']['ID_Person'] == p:
            if len(s['portalData']['people_Groups']) > 0:
                for i in s['portalData']['people_Groups']:
                    if i.get('people_Groups::Group') in ['Governance Advisory Board', 'International Science Advisory Panel', 'Program Executive Group']:
                        row = [ active_ppl_saef[p]['FirstName'], active_ppl_saef[p]['LastName'], active_ppl_saef[p]['Gender'], i.get('people_Groups::Group') ]
                        positions.append(row)


In [201]:
df = pd.DataFrame(positions, columns=['FirstName', 'LastName','Gender', 'Position'])



In [202]:
df.groupby(['Position', 'Gender'])['Gender'].count()

Position                              Gender               
Governance Advisory Board             Man                       4
                                      Woman                     4
International Science Advisory Panel  Man                       6
                                      Woman                     8
Program Executive Group               Man                       5
                                      Woman                     9
Program Staff                         Man                       2
                                      Unspecified diversity     1
                                      Woman                     7
Researcher                            Man                      54
                                      Unspecified diversity     3
                                      Woman                    63
Name: Gender, dtype: int64